# Simple modal to predict if machine is working or not, after given number of hours it has been active

The training data should have labels of active working hours of similar machine and wether its working or not.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
# Dataset (feature: hours used)
# Labels: [0 = broken, 1 = working]
data = torch.tensor([[200], 
                     [1500], 
                     [800], 
                     [1200], 
                     [400]],
                   dtype=torch.float32)

labels = torch.tensor([1, 0, 1, 0, 1], dtype=torch.float32).view(-1, 1) 
# Checking the shapes of data and labels
print("Data:", data)
print("Labels:", labels)


Data: tensor([[ 200.],
        [1500.],
        [ 800.],
        [1200.],
        [ 400.]])
Labels: tensor([[1.],
        [0.],
        [1.],
        [0.],
        [1.]])


# Defining a simple model with Linear Layer, Activation, and Layer Normalization

In [6]:
class MachineStatusModel(nn.Module):
    def __init__(self):
        super(MachineStatusModel, self).__init__()
        
        # the linear layer
        # Input size = 1 (hours used) and the output size = 1 (probability of working)
        self.linear = nn.Linear(1, 1)
        
        # Layer normalization and is applied after linear layer
        self.layer_norm = nn.LayerNorm(1)
        
        # This is Sigmoid activation function 
        #Is used for binary classification : (0 = broken, 1 = working)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Data flows through the network
        # Linear transformation: W*x + b
        x = self.linear(x)
        
        # Apply Layer Normalization
        x = self.layer_norm(x)
        
        # Apply Sigmoid to get the probability of machine working (between 0 and 1)
        x = self.sigmoid(x)
        
        return x

# Instantiating the model, loss function, and optimizer

In [5]:
model = MachineStatusModel()
# Binary Cross-Entropy Loss (since this is a binary classification problem)
criterion = nn.BCELoss()
# Stochastic Gradient Descent (SGD) for optimization
optimizer = optim.SGD(model.parameters(), lr=0.01)


# Training the model with given data

In [ ]:
epochs = 1000
for epoch in range(epochs):
    # Forward pass: compute the model output (probability of working)
    predictions = model(data)
    
    # Compute the loss (binary cross-entropy loss)
    loss = criterion(predictions, labels)
    
    # Backward pass: compute gradients
    # Clear previous gradients
    optimizer.zero_grad()  
    # New gradients
    loss.backward()       
    
    # Update weights using the optimizer
    optimizer.step()
    
    # Print the loss every 100 epochs to track progress
    if (epoch + 1) % 100 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")


Epoch [100/1000], Loss: 0.6853
Epoch [200/1000], Loss: 0.6805
Epoch [300/1000], Loss: 0.6776
Epoch [400/1000], Loss: 0.6758
Epoch [500/1000], Loss: 0.6747
Epoch [600/1000], Loss: 0.6741
Epoch [700/1000], Loss: 0.6737
Epoch [800/1000], Loss: 0.6734
Epoch [900/1000], Loss: 0.6733
Epoch [1000/1000], Loss: 0.6732


# Evaluating the trained model

to get an output from a simple input

In [ ]:

with torch.no_grad():
    # Now let's predict
    new_machine = torch.tensor([[900]], dtype=torch.float32)
    predicted_prob = model(new_machine)
    
    # make it binary...into true or false
    predicted_class = (predicted_prob > 0.5).float() 
    
    print(f"\nPredicted probability of working for 900 hours of usage: {predicted_prob.item():.4f}")
    print(f"Predicted class (0=Broken, 1=Working): {predicted_class.item()}")


Predicted probability of working for 900 hours of usage: 0.5914
Predicted class (0=Broken, 1=Working): 1.0
